## Welcome to Stockastic Piraites! 🏴‍☠️🏴‍☠️🏴‍☠️ 🦜

### Libraries versions
Correct version of torch is important!
- pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu121
- pip3 install transformers==4.31.0
- pip install ctransformers
- pip install pydub

In [1]:
import torch
print(torch.__version__)
# Check if CUDA is available
cuda_available = torch.cuda.is_available()

# Print the result
if cuda_available:
    print("CUDA is available in your PyTorch installation.")
    print("CUDA version:", torch.version.cuda)
else:
    print("CUDA is not available in your PyTorch installation.")

2.3.0.dev20231218+cu121
CUDA is available in your PyTorch installation.
CUDA version: 12.1


In [3]:
import utils

# Replace 'your_pid_here' with the actual PID of the process you want to stop
#process_pid = 1175331

# Call the function to stop the process
#stop_process(process_pid)


In [4]:
#Restart Kernel
restart = True
if restart:
    import IPython
    #IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel
    IPython.Application.instance().kernel.do_restart() #does not automatically restart kernel

In [2]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices
from ctransformers import AutoModelForCausalLM

#preset super_fast, fast, standard, high_quality
preset = "fast"

# This will download all the models used by Tortoise from the HF hub.
tts = TextToSpeech(use_deepspeed=False, kv_cache=True, device='cuda')

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = AutoModelForCausalLM.from_pretrained("/workspace/data/mistral_model/mistral-7b-instruct-v0.1.Q4_K_M.gguf", model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf", model_type="mistral", gpu_layers=50)

/root/miniconda/envs/tortoise/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at jbetker/wav2vec2-large-robust-ft-libritts-voxpopuli were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were

In [3]:
import datetime

def get_day_of_week():
    # Get the current date and time
    current_datetime = datetime.datetime.now()

    # Use the strftime method to format the date and extract the day of the week
    day_of_week = current_datetime.strftime("%A")

    return day_of_week

# Call the function and print the result
day = get_day_of_week()
print("Today is", day)

Today is Monday


In [4]:
#Create a list of topics
generalTopics = ["God", "chat G.P.T. religion", "quantum physics", "A.I. will make human useless", "descoveries in the human brain", "close the program"]

listOFTopics = ["They talk about if god exist or not, for instance if God created the universe and god is omniscient, omnipotent and omnibenevolent, why is needed a simulation? make it funny",
                "They talk about chat G.P.T. inventing a new religion that all humans are going to follow blindly",
                "They explain quantum physics for advanced audience, use complex terms",
                "They talk about A.I. will make humans useless, for instance, if A.I. can do everything, why do we need humans?",
                "They talk about recent descoveries in the human brain, make it funny",
                "They close the program, make it funny"]

print("number of topics", len(listOFTopics))
preTopic = "Generate a radio conversation of two commentators, Anna and Phillip. "
topic = ""
postTopic = ". Don't bring more people into the conversation, only two voices. Only refers to them with their names, avoid adding time or other elements."

number of topics 6


In [ ]:
#Generate the conversations and save them in the data folder
from tqdm import tqdm

mistral_generation = ""
changeTopic = ""
day = get_day_of_week()

for i in tqdm(range(len(listOFTopics))):
    print(i, "\n--------------------------------------")
    if i == 0:
        changeTopic = "Start with greetings too all the audience, knowing that today is " + day + ". "
    else:
        changeTopic = "No greetings, don't say hey. They were talking about " + generalTopics[i-1] + " and now they change to " + generalTopics[i] + ". "
    prompt =  preTopic + changeTopic + listOFTopics[i] + postTopic
    print(prompt+"\n")
    generate=llm(prompt)
    #print(generate, "\n")
    mistral_generation += generate

In [ ]:
#Fix some text errors 
replacements = {
    "Anna:": "\nAnna:",
    "Phillip:": "\nPhillip:",
    "(laughing)": "hahahaha",
    "(laugh)": "hahaha",
    "(laughs)": "hahaha",
    "(smiling)": "hahaa",
    "(smile)": "hahaha",
    "(smiles)": "hahaha",
    "(chuckles)": "haha",
    "\n\n": "\n",
}
#Fix expressions 
final_text = mistral_generation
for old, new in replacements.items():
    final_text = final_text.replace(old, new)

final_text = final_text.replace("\n\n", "\n")
# Split the text into lines and get the first line
lines = final_text.splitlines()

# Check if the first line starts with "Anna:" or "Phillip:"
if lines and (lines[0].startswith("Anna:") or lines[0].startswith("Phillip:")):
    # Print the first line
    print("First line error fixed:", lines[0])
else:
    # Erase the first line
    lines.pop(0)

# Print the modified text (without the erased line)
final_text = '\n'.join(lines)

#print(final_text)

In [12]:
def generateVoice(voice, text, preset, fileName):
    # Pick one of the voices from the output above
    # Load it and send it through Tortoise.
    voice_samples, conditioning_latents = load_voice(voice)
    gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset=preset)
    torchaudio.save(fileName, gen.squeeze(0).cpu(), 24000)
    print("Saved to " + fileName)
    #IPython.display.Audio(fileName)

In [13]:
from datetime import datetime

# Get the current date and time
def getCurrentTime():
    current_datetime = datetime.now()

    # Extract the date, hour, minute, and second components
    current_date = current_datetime.strftime("%d%m%y")
    current_hour = current_datetime.hour
    current_minute = current_datetime.minute
    current_second = current_datetime.second

    # Calculate the current second of the day
    current_second_of_day = current_hour * 3600 + current_minute * 60 + current_second

    # Create a unique ID for naming files
    file_id = f"{current_second_of_day}_{current_date}"
    return file_id

In [14]:
#generate the audio samples
def audioGeneration(voice, text, preset):
    print("----------------------------")
    print("voice presset:", voice, "\n", text, "\n")
    path = "../workspace/data/presenters/"
    time = getCurrentTime()
    fileName =  path + time +"_"+ voice + '.wav'
    generateVoice(voice, text, preset, fileName)

In [15]:
import os

# Specify the path to the 'data' folder
folder_path = '/workspace/data/presenters'

# Check if the folder exists before attempting to remove its contents
if os.path.exists(folder_path):
    # Get a list of all files in the 'data' folder
    files = os.listdir(folder_path)
    print(f"Removing {len(files)} files from {folder_path}...")
    # Iterate through the files and remove each one
    for file in files:
        file_path = os.path.join(folder_path, file)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
                print(f"Successfully deleted {file_path}")
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")
else:
    # If the folder doesn't exist, create it
    os.makedirs(folder_path)


Removing 15 files from /workspace/data/presenters...
Successfully deleted /workspace/data/presenters/45724_181223_emma.wav
Successfully deleted /workspace/data/presenters/45797_181223_tom.wav
Successfully deleted /workspace/data/presenters/45857_181223_emma.wav
Successfully deleted /workspace/data/presenters/45957_181223_tom.wav
Successfully deleted /workspace/data/presenters/46052_181223_emma.wav
Successfully deleted /workspace/data/presenters/46129_181223_emma.wav
Successfully deleted /workspace/data/presenters/46166_181223_tom.wav
Successfully deleted /workspace/data/presenters/46248_181223_emma.wav
Successfully deleted /workspace/data/presenters/46276_181223_tom.wav
Successfully deleted /workspace/data/presenters/46363_181223_emma.wav
Successfully deleted /workspace/data/presenters/46403_181223_tom.wav
Successfully deleted /workspace/data/presenters/46523_181223_emma.wav
Successfully deleted /workspace/data/presenters/46553_181223_emma.wav
Successfully deleted /workspace/data/prese

In [ ]:
#Split text into two sections dictionary, one for Anna and one for Phillip
lines = final_text.split('\n')

# Flag to determine the current speaker
current_speaker = None

# Iterate through lines and populate dictionaries
for line in lines:
    if line.startswith("Anna:"):
        current_speaker = "angie" #this is the voice name in the dataset
        text = line[len("anna:"):].strip()
        audioGeneration(current_speaker, text, preset)
        
    elif line.startswith("Phillip:"):
        current_speaker = "tom" #this is the voice name in the dataset
        text = line[len("phillip:"):].strip()
        audioGeneration(current_speaker, text, preset)

In [ ]:
import os

folder_path = "/workspace/data/presenters/"

# Get a list of all files in the folder
file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Print the list of file names
# print("File names in the folder:")
# for file_name in file_names:
#     print(file_name)

In [19]:
from pydub import AudioSegment
from pydub.effects import normalize

sourceFolder = "../workspace/data/rendered_audio/"

audio_segment = AudioSegment.silent(duration=0)
# Create an empty audio segment to hold the concatenated audio
concatenated_audio = AudioSegment.silent(duration=4000)

# Iterate through each audio file and concatenate it
for file in file_names:
    audio_segment = AudioSegment.from_file(folder_path+file, format="wav")
    # Normalize the audio to increase volume
    audio_segment = normalize(audio_segment)
    concatenated_audio += audio_segment

# Export the concatenated audio to a new file
concatenated_audio.export(sourceFolder+"concatenated_output_1.mp3", format="wav")

<_io.BufferedRandom name='../workspace/data/rendered_audio/concatenated_output_1.mp3'>

In [20]:
pathBackground = "../workspace/data/audio_background/"

# Load your audio tracks
track0 = AudioSegment.from_file(pathBackground+"opening_news.mp3", format="mp3")
track1 = AudioSegment.from_file(pathBackground+"Pop.mp3", format="mp3")
track2 = AudioSegment.from_file(pathBackground+"japanese_pop.mp3", format="mp3")
track3 = AudioSegment.from_file(pathBackground+"cumbia_3.mp3", format="mp3")

# Add the tracks together
combined_track = track0 + track1 + track2 + track3 + track0

# Reduce the volume to half with a ramp of 1 second after 2 seconds
ramp_duration = 2000  # 1 second in milliseconds
start_time = 2000  # 2 seconds in milliseconds

# Extract the part of the audio before the volume reduction
audio_before_ramp = combined_track[:start_time]

# Extract the part of the audio during the volume reduction and apply fade-out
audio_ramp = combined_track[start_time:start_time + ramp_duration].fade_out(ramp_duration)

# Extract the part of the audio after the volume reduction
audio_after_ramp = combined_track[start_time + ramp_duration:]

# Decrease the gain by 12 dB to reduce volume to half
audio_half_volume = audio_after_ramp - 12

# Combine the audio segments
final_audio = audio_before_ramp + audio_ramp + audio_half_volume 

# Export the modified audio to a new file
final_audio.export(pathBackground+"new_background.mp3", format="mp3")

<_io.BufferedRandom name='../workspace/data/audio_background/new_background.mp3'>

In [ ]:
# Mix the background music with the conversation
background_music = AudioSegment.from_file(pathBackground+"new_background.mp3", format="mp3")

mixed_audio = concatenated_audio.overlay(background_music)  # Add more overlay calls for additional tracks
# Export the mixed audio to a new file
time = getCurrentTime()
fileName = sourceFolder+time+"_mixed_output.mp3"
mixed_audio.export(fileName, format="wav")
print(fileName)
